In [1]:
#Installing PyTube
! pip install pytube -q 

     |████████████████████████████████| 56 kB 4.7 MB/s 


In [2]:
#importing YouTube library from PyTube
from pytube import YouTube

In [3]:
#importing video from it's URL
VIDEO_URL = 'https://www.youtube.com/watch?v=lO36Q8Uj2bE&ab_channel=FIFA' 

In [4]:
yt = YouTube(VIDEO_URL)

In [5]:
yt.streams \
  .filter(only_audio = True, file_extension = 'mp4') \
  .first() \
  .download(filename = 'ytaudio.mp4') \

'/content/ytaudio.mp4'

In [6]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [7]:
!pip install huggingsound -q 

     |████████████████████████████████| 214 kB 54.4 MB/s 
     |████████████████████████████████| 5.5 MB 51.0 MB/s 
     |████████████████████████████████| 451 kB 71.0 MB/s 
     |████████████████████████████████| 182 kB 56.5 MB/s 
     |████████████████████████████████| 115 kB 88.8 MB/s 
     |████████████████████████████████| 212 kB 81.1 MB/s 
     |████████████████████████████████| 1.4 MB 62.7 MB/s 
     |████████████████████████████████| 2.2 MB 63.7 MB/s 
     |████████████████████████████████| 127 kB 62.3 MB/s 
     |████████████████████████████████| 7.6 MB 59.3 MB/s 


In [8]:
from huggingsound import SpeechRecognitionModel

In [9]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
device 

'cpu'

In [11]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)


INFO:huggingsound.speech_recognition.model:Loading model...


Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/262 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [12]:
import librosa

In [13]:
input_file = '/content/ytaudio.wav'

In [14]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [15]:
import soundfile as sf


In [16]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

In [17]:
i

19

In [18]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav') 

In [19]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 20/20 [15:35<00:00, 46.77s/it]


In [20]:
full_transcript = ' '
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

  len(full_transcript)

In [21]:
#importing pipeline from transformers
from transformers import pipeline

In [22]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [23]:
summarized_text = summarization(full_transcript)

In [24]:
summarized_text[0]['summary_text']

' Both teams have missed their opening penalty and scored the next twobussexshurnathirty-two years of ch played in the heredagazyn irelevents for his entire career . Both sides have taken four penalties in the shootout both have only scored twice and it now is suddain death .'

In [25]:
num_iters = int(len(full_transcript)/1000)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i + 1) * 1000
  #print("input text \n" + full_transcript[start:end])
  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
 # print("Summarized text\n"+out)
  summarized_text.append(out)

#print(summarized_text)

Your max_length is set to 20, but you input_length is only 13. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)


In [26]:
print("Summarized text\n"+out)

Summarized text
 amantic penalty shoters you are likely to ever see are some of the best ever
